In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Flatten, Reshape
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras import regularizers, metrics
import numpy as np
import keras.layers
import keras.losses

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import sklearn

Using TensorFlow backend.


In [2]:
(xtrain, ytrain), (xtest, ytest) = mnist.load_data()
# xtrain=xtrain()
maxV = float(xtrain.max())
xtrain=xtrain.astype('float32')/maxV
xtrain=np.reshape(xtrain, (len(xtrain), np.prod(xtrain.shape[1:])))

xtest=xtest.astype('float32')/maxV
xtest=np.reshape(xtest, (len(xtest), np.prod(xtest.shape[1:])))
# print(xtest[0].shape)

In [3]:
digits=list(set(ytrain[:100]))
count=0
index=0
arr=[]
lab=[]
for d in digits:
    count=0
    index=0
    while count<100:
        if(ytrain[index]==d):
            arr.append(xtrain[index])
            lab.append(ytrain[index])
            count+=1
        index+=1
arr=np.asarray(arr)
lab=np.asarray(lab)
print(arr.shape)
print(lab.shape)

(1000, 784)
(1000,)


In [4]:
##As we are not reconstructing, I only created an encoder module
N=[2, 5, 10]

model1=Sequential()
model1.add(Dense(N[0], input_dim=arr.shape[1], activation='relu'))

model2=Sequential()
model2.add(Dense(N[1], input_dim=arr.shape[1], activation='relu'))

model3=Sequential()
model3.add(Dense(N[2], input_dim=arr.shape[1], activation='relu'))

In [5]:
##Encoder in action
model1_predict = model1.predict(arr)
model2_predict = model2.predict(arr)
model3_predict = model3.predict(arr)

In [6]:
print(model1_predict.shape)

(1000, 2)


In [7]:
##KNN classifier for 50 neighbours.

k=50
knn_1 = KNeighborsClassifier(n_neighbors=k)
knn_2 = KNeighborsClassifier(n_neighbors=k)
knn_3 = KNeighborsClassifier(n_neighbors=k)

knn_1.fit(model1_predict, lab)
knn_2.fit(model2_predict, lab)
knn_3.fit(model3_predict, lab)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=50, p=2,
           weights='uniform')

In [8]:
##Run KNN on all inputs.
pred_lab_1=[]
pred_lab_2=[]
pred_lab_3=[]
pred_1_score=0
pred_2_score=0
pred_3_score=0

for i in range(len(model1_predict)):
    pred_lab_1.append(knn_1.predict([model1_predict[i]])[0])
    pred_lab_2.append(knn_2.predict([model2_predict[i]])[0])
    pred_lab_3.append(knn_3.predict([model3_predict[i]])[0])
    
pred_lab_1=np.asarray(pred_lab_1)
pred_lab_2=np.asarray(pred_lab_2)
pred_lab_3=np.asarray(pred_lab_3)

pred_1_score=knn_1.score(model1_predict, lab)
pred_2_score=knn_2.score(model2_predict, lab)
pred_3_score=knn_3.score(model3_predict, lab)
print("Accuracy of N:2 -> ",pred_1_score*100, "%")
print("Accuracy of N:5 -> ",pred_2_score*100, "%")
print("Accuracy of N:10 -> ",pred_3_score*100, "%")

acc=[1-pred_1_score, 1-pred_2_score, 1-pred_3_score]

Accuracy of N:2 ->  22.0 %
Accuracy of N:5 ->  36.199999999999996 %
Accuracy of N:10 ->  44.2 %


In [9]:
print("\n\t\t\tScore -> N = 2\n")
print(classification_report(lab, pred_lab_1))


			Score -> N = 2

              precision    recall  f1-score   support

           0       0.42      0.22      0.29       100
           1       0.16      0.43      0.23       100
           2       0.00      0.00      0.00       100
           3       0.19      0.33      0.24       100
           4       0.28      0.42      0.33       100
           5       0.22      0.04      0.07       100
           6       0.09      0.03      0.05       100
           7       0.28      0.59      0.38       100
           8       0.11      0.05      0.07       100
           9       0.26      0.09      0.13       100

   micro avg       0.22      0.22      0.22      1000
   macro avg       0.20      0.22      0.18      1000
weighted avg       0.20      0.22      0.18      1000



In [10]:
print("\n\t\t\tScore -> N = 5\n")
print(classification_report(lab, pred_lab_2))


			Score -> N = 5

              precision    recall  f1-score   support

           0       0.41      0.41      0.41       100
           1       0.35      0.48      0.40       100
           2       0.56      0.18      0.27       100
           3       0.37      0.15      0.21       100
           4       0.26      0.23      0.24       100
           5       0.19      0.03      0.05       100
           6       0.39      0.55      0.46       100
           7       0.34      0.52      0.41       100
           8       0.50      0.63      0.56       100
           9       0.26      0.44      0.33       100

   micro avg       0.36      0.36      0.36      1000
   macro avg       0.36      0.36      0.34      1000
weighted avg       0.36      0.36      0.34      1000



In [11]:
print("\n\t\t\tScore -> N = 10\n")
print(classification_report(lab, pred_lab_3))


			Score -> N = 10

              precision    recall  f1-score   support

           0       0.45      0.39      0.42       100
           1       0.35      0.94      0.51       100
           2       0.62      0.15      0.24       100
           3       0.52      0.29      0.37       100
           4       0.59      0.55      0.57       100
           5       0.44      0.31      0.36       100
           6       0.40      0.66      0.50       100
           7       0.54      0.53      0.54       100
           8       0.43      0.20      0.27       100
           9       0.42      0.40      0.41       100

   micro avg       0.44      0.44      0.44      1000
   macro avg       0.48      0.44      0.42      1000
weighted avg       0.48      0.44      0.42      1000



In [17]:
plt.figure()
plt.plot(N, acc)
plt.scatter(N, acc, c='red')
plt.title('Mis Classification Error Plot - Autoencoder Model')
plt.ylabel('error')
plt.xlabel('N')
plt.legend(['K-NN'], loc='upper right')
plt.show()

In [20]:
print(acc)

#Borrowing metrics from PCA
pca_acc=[0.502, 0.257, 0.18600000000000005]

[0.78, 0.638, 0.558]


In [22]:
plt.figure()
plt.plot(N, acc)
plt.plot(N, pca_acc)
plt.scatter(N, acc, c='red')
plt.scatter(N, pca_acc, c='blue')
plt.title('Mis Classification Error Plot')
plt.ylabel('error')
plt.xlabel('N')
plt.legend(['AutoEncoder', 'PCA'], loc='upper right')
plt.show()